In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
import gc
import lightgbm as lgb
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from scipy.stats import norm, probplot, skew
from scipy.special import boxcox1p

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, PolynomialFeatures, RobustScaler
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import KFold, GridSearchCV, train_test_split, cross_val_score
from sklearn.feature_selection import SelectKBest, f_classif, chi2
from sklearn.metrics import mean_squared_error as mse, mean_absolute_error as mae
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor as rfr, GradientBoostingRegressor as gbr
from sklearn.svm import SVR

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from mlxtend.regressor import StackingCVRegressor

from functools import partial

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/빅데이터 분석 실습/[실전 과제] Kaggle playground/train.csv')
test = pd.read_csv('/content/drive/MyDrive/빅데이터 분석 실습/[실전 과제] Kaggle playground/test.csv')

In [ ]:
train.head()

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,...,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,target
0,0,0.106643,3.59437,132.8040,3.18428,0.081971,1.18859,3.73238,2.266270,2.09959,0.012330,1.607190,-0.318058,0.560137,2.806880,1.35114,2.535930,0.197527,0.676494,1.98979,-3.842450,0.037380,0.230322,3.33055,0.009397,0.144738,3.05131,1.30362,0.033225,-0.018284,2.748210,-0.009294,-0.036271,-0.049871,0.019484,3.898460,11.2863,1.138020,3.366880,4.94446,...,-0.027551,0.019483,-0.048826,0.050748,3.729300,5.017440,4.186880,0.063342,0.121043,1.37175,4.017450,0.167613,0.039754,2.042360,-0.016614,0.107679,3.507250,0.013660,-0.097023,5.396070,0.244457,3.49184,0.113090,-0.015472,4.208790,4.106560,0.037227,-0.118814,0.067086,0.010739,1.09862,0.013331,-0.011715,0.052759,0.065400,4.211250,1.97877,0.085974,0.240496,0
1,1,0.125021,1.67336,76.5336,3.37825,0.099400,5.09366,1.27562,-0.471318,4.54594,0.037706,0.331749,0.325091,0.062040,2.262150,4.33943,-0.224999,0.233586,3.381280,1.90299,0.067874,-0.051268,0.006135,2.60444,0.103441,0.067638,4.75362,1.85552,-0.181834,0.008359,3.166340,0.011850,0.022292,0.069320,0.117109,0.315276,24.4807,1.672270,-0.409067,4.95475,...,-0.010841,0.064584,0.102548,0.093611,0.964089,0.630422,4.307340,0.091289,-0.036360,3.61767,3.103240,0.000657,0.051302,1.924620,0.123294,-0.022671,1.548120,-0.010397,0.058330,3.661310,-0.118386,2.35739,-0.009112,0.178701,4.097350,3.532890,0.005244,0.121381,0.109968,0.135838,3.46017,0.017054,0.124863,0.154064,0.606848,-0.267928,2.57786,-0.020877,0.024719,0
2,2,0.036330,1.49747,233.5460,2.19435,0.026914,3.12694,5.05687,3.849460,1.80187,0.056995,0.328684,2.968810,0.105244,2.069490,5.30986,1.354790,-0.262018,1.379080,1.48091,0.020542,-0.008806,0.109348,1.68365,0.038180,0.123716,1.11248,3.57166,0.120601,0.082069,2.233520,0.002270,0.045182,0.014405,0.011599,-0.502849,33.7382,1.417500,1.071350,3.22296,...,0.027571,-0.007121,-0.048914,-0.002574,1.865090,2.404170,0.411741,0.057749,0.525174,2.16879,0.828297,0.089848,0.093744,4.949010,-0.010978,0.076671,0.266784,0.038691,0.382731,3.847600,-0.121482,3.74084,0.147098,-0.016566,0.614651,2.125840,0.078828,0.979808,0.026758,0.117310,4.88300,0.085222,0.032396,0.116092,-0.001689,-0.520069,2.14112,0.124464,0.148209,0
3,3,-0.014077,0.24600,779.9670,1.89064,0.006948,1.53112,2.69800,4.517330,4.50332,0.123494,1.002680,4.869600,0.058411,2.497850,1.23843,2.348360,0.175475,1.608890,2.02881,0.042086,0.005141,0.076506,1.65122,0.111813,0.121641,0.58912,4.23692,-0.032843,0.058168,0.712927,0.097465,0.072744,0.000324,0.063362,4.063820,25.3824,0.576572,2.026210,2.96843,...,0.110884,0.026837,2.931160,0.068112,-0.495192,1.345280,2.242750,0.035611,-0.139274,4.74243,3.292740,0.117877,0.065605,0.556711,-0.058029,0.070501,1.101250,0.068559,0.162928,4.070180,-0.008835,3.89678,0.913739,-0.163204,3.074850,4.356780,-0.048894,4.917990,0.069930,-0.015347,3.47439,-0.017103,-0.008100,0.062013,0.041193,0.511657,1.96860,0.040017,0.044873,0
4,4,-0.003259,3.71542,156.1280,2.14772,0.018284,2.09859,4.15492,-0.038236,3.37145,0.034166,0.711483,0.769988,0.057555,0.957257,3.71145,5.464350,0.287104,2.616950,1.38403,0.074883,-0.010543,0.109121,2.27602,0.008023,0.045236,4.35954,5.07562,-0.009376,0.528966,4.053350,0.020000,0.106828,0.051307,0.045939,3.402460,15.5615,1.635960,0.047029,4.01771,...,0.075586,0.032114,-0.042284,0.047974,-0.294184,5.065600,1.050290,0.034019,0.024611,3.12578,2.262840,0.082462,-0.023296,5.615850,0.086238,0.157568,3.725670,0.061247,0.086603,0.607246,1.411090,2.06062,-0.023154,0.011234,2.155530,0.914518,0.044521,0.375731,0.134351,0.013781,1.91059,-0.042943,0.105616,0.125072,0.037509,1.043790,1.07481,-0.012819,0.072798,1


In [ ]:
train.shape

(600000, 102)

In [ ]:
train.isnull().sum().sum()

0

In [ ]:
train.drop('id',axis=1,inplace=True)
test.drop('id',axis=1,inplace=True)

In [ ]:
from numpy.random import seed
seed(42)
import tensorflow as tf
tf.random.set_seed(42)
from tensorflow import keras
import numpy as np
from keras import backend as K
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=20, verbose=0,
    mode='min',restore_best_weights=True)


plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=5, verbose=0,
    mode='min')

In [ ]:
colNames=[col for col in test.columns if col not in ['id','f2','f35','f44']]
# colNames.remove('id')
train.replace([np.inf, -np.inf], np.nan,inplace=True)
test.replace([np.inf, -np.inf], np.nan,inplace=True)
train_nn=train[colNames].copy()
test_nn=test[colNames].copy()
mm = MinMaxScaler()
mm.fit(train[['f2','f35','f44']])
train_scaled = mm.transform(train[['f2','f35','f44']])
train_scaled = pd.DataFrame(data = train_scaled, columns = ['f2','f35','f44'])
mm.fit(test[['f2','f35','f44']])
test_scaled = mm.transform(test[['f2','f35','f44']])
test_scaled = pd.DataFrame(data = test_scaled, columns = ['f2','f35','f44'])
train_nn = pd.concat([train_nn, train_scaled], axis = 1)
test_nn = pd.concat([test_nn, test_scaled], axis = 1)
train_nn=pd.concat([train_nn,test_nn])

In [ ]:
train_nn.shape

(1140000, 100)

In [ ]:
train.shape

(600000, 101)

In [ ]:
test.shape

(540000, 100)

In [ ]:
train_nn['target']=train['target']

In [ ]:
from keras.backend import sigmoid
from sklearn.metrics import roc_auc_score
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
get_custom_objects().update({'swish': Activation(swish)})

hidden_units = (128,64,32)
#initialized number of neurons in each hidden layer

def base_model():

    num_input = keras.Input(shape=(test.shape[1],), name='num_data')#input layer


    out = keras.layers.Concatenate()([num_input])
    
    # Add one or more hidden layers
    for n_hidden in hidden_units:

        out = keras.layers.Dense(n_hidden, activation='softmax')(out)
        out = keras.layers.Dropout(rate = 0.2)(out)
    #out = keras.layers.Concatenate()([out, num_input])

    # A single output: our predicted target value probability
    out = keras.layers.Dense(1, activation='sigmoid', name='prediction')(out)
    
    model = keras.Model(
    inputs = [num_input],
    outputs = out,
    )
    
    return model

In [ ]:
train_nn

,f0,f1,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f36,f37,f38,f39,f40,f41,...,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f2,f35,f44,target
0,0.106643,3.59437,3.184280,0.081971,1.188590,3.732380,2.266270,2.09959,0.012330,1.607190,-0.318058,0.560137,2.806880,1.35114,2.535930,0.197527,0.676494,1.98979,-3.842450,0.037380,0.230322,3.330550,0.009397,0.144738,3.051310,1.303620,0.033225,-0.018284,2.748210,-0.009294,-0.036271,-0.049871,0.019484,3.898460,1.138020,3.366880,4.94446,-0.105772,2.11345,3.452230,...,0.050748,3.729300,5.017440,4.186880,0.063342,0.121043,1.37175,4.017450,0.167613,0.039754,2.042360,-0.016614,0.107679,3.507250,0.013660,-0.097023,5.396070,0.244457,3.49184,0.113090,-0.015472,4.208790,4.106560,0.037227,-0.118814,0.067086,0.010739,1.098620,0.013331,-0.011715,0.052759,0.065400,4.211250,1.97877,0.085974,0.240496,0.248101,0.219337,0.252487,0
1,0.125021,1.67336,3.378250,0.099400,5.093660,1.275620,-0.471318,4.54594,0.037706,0.331749,0.325091,0.062040,2.262150,4.33943,-0.224999,0.233586,3.381280,1.90299,0.067874,-0.051268,0.006135,2.604440,0.103441,0.067638,4.753620,1.855520,-0.181834,0.008359,3.166340,0.011850,0.022292,0.069320,0.117109,0.315276,1.672270,-0.409067,4.95475,0.092358,2.60318,1.954690,...,0.093611,0.964089,0.630422,4.307340,0.091289,-0.036360,3.61767,3.103240,0.000657,0.051302,1.924620,0.123294,-0.022671,1.548120,-0.010397,0.058330,3.661310,-0.118386,2.35739,-0.009112,0.178701,4.097350,3.532890,0.005244,0.121381,0.109968,0.135838,3.460170,0.017054,0.124863,0.154064,0.606848,-0.267928,2.57786,-0.020877,0.024719,0.241034,0.226425,0.258653,0
2,0.036330,1.49747,2.194350,0.026914,3.126940,5.056870,3.849460,1.80187,0.056995,0.328684,2.968810,0.105244,2.069490,5.30986,1.354790,-0.262018,1.379080,1.48091,0.020542,-0.008806,0.109348,1.683650,0.038180,0.123716,1.112480,3.571660,0.120601,0.082069,2.233520,0.002270,0.045182,0.014405,0.011599,-0.502849,1.417500,1.071350,3.22296,2.122030,3.08216,0.637555,...,-0.002574,1.865090,2.404170,0.411741,0.057749,0.525174,2.16879,0.828297,0.089848,0.093744,4.949010,-0.010978,0.076671,0.266784,0.038691,0.382731,3.847600,-0.121482,3.74084,0.147098,-0.016566,0.614651,2.125840,0.078828,0.979808,0.026758,0.117310,4.883000,0.085222,0.032396,0.116092,-0.001689,-0.520069,2.14112,0.124464,0.148209,0.260754,0.231398,0.343598,0
3,-0.014077,0.24600,1.890640,0.006948,1.531120,2.698000,4.517330,4.50332,0.123494,1.002680,4.869600,0.058411,2.497850,1.23843,2.348360,0.175475,1.608890,2.02881,0.042086,0.005141,0.076506,1.651220,0.111813,0.121641,0.589120,4.236920,-0.032843,0.058168,0.712927,0.097465,0.072744,0.000324,0.063362,4.063820,0.576572,2.026210,2.96843,1.085670,1.71088,1.371820,...,0.068112,-0.495192,1.345280,2.242750,0.035611,-0.139274,4.74243,3.292740,0.117877,0.065605,0.556711,-0.058029,0.070501,1.101250,0.068559,0.162928,4.070180,-0.008835,3.89678,0.913739,-0.163204,3.074850,4.356780,-0.048894,4.917990,0.069930,-0.015347,3.474390,-0.017103,-0.008100,0.062013,0.041193,0.511657,1.96860,0.040017,0.044873,0.329385,0.226910,0.654519,0
4,-0.003259,3.71542,2.147720,0.018284,2.098590,4.154920,-0.038236,3.37145,0.034166,0.711483,0.769988,0.057555,0.957257,3.71145,5.464350,0.287104,2.616950,1.38403,0.074883,-0.010543,0.109121,2.276020,0.008023,0.045236,4.359540,5.075620,-0.009376,0.528966,4.053350,0.020000,0.106828,0.051307,0.045939,3.402460,1.635960,0.047029,4.01771,0.155748,5.28998,4.118920,...,0.047974,-0.294184,5.065600,1.050290,0.034019,0.024611,3.12578,2.262840,0.082462,-0.023296,5.615850,0.086238,0.157568,3.725670,0.061247,0.086603,0.607246,1.411090,2.06062,-0.023154,0.011234,2.155530,0.914518,0.044521,0.375731,0.134351,0.013781,1.910590,-0.042943,0.105616,0.125072,0.037509,1.043790,1.07481,-0.012819,0.072798,0.251031,0.221634,0.262139,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [ ]:
target_name='target'
scores_folds = {}
model_name = 'NN'
pred_name = 'pred_{}'.format(model_name)

n_folds = 5
kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=2020)
scores_folds[model_name] = []
counter = 1

features_to_consider = list(train_nn)

features_to_consider.remove('target')
try:
    features_to_consider.remove('pred_NN')
except:
    pass


train_nn[features_to_consider] = train_nn[features_to_consider].fillna(train_nn[features_to_consider].mean())
test_nn[features_to_consider] = test_nn[features_to_consider].fillna(train_nn[features_to_consider].mean())

train_nn[pred_name] = 0
test_nn[target_name] = 0
test_predictions_nn = np.zeros(test_nn.shape[0])

for fold, (trn_ind, val_ind) in enumerate(kf.split(train,train['target'])):
    print(f'Training fold {fold + 1}')
    X_train, X_test = train_nn.iloc[trn_ind][features_to_consider], train_nn.iloc[val_ind][features_to_consider]
    y_train, y_test = train_nn.iloc[trn_ind]['target'], train_nn.iloc[val_ind]['target']
    print('CV {}/{}'.format(counter, n_folds)) 
    #############################################################################################
    # NN
    #############################################################################################
    print(X_train.shape,y_train.shape)
    print(X_test.shape, y_test.shape)
    model = base_model()
    
    model.compile(
        keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics = ['AUC']
    )
    
    num_data = X_train[features_to_consider]
    
    scaler = MinMaxScaler(feature_range=(0, 1))         
    num_data = scaler.fit_transform(num_data.values)    
      
    target =  y_train
    
    num_data_test = X_test[features_to_consider]
    num_data_test = scaler.transform(num_data_test.values)

    model.fit([num_data], 
              target,               
              batch_size=2048,
              epochs=1000,
              validation_data=([num_data_test], y_test),
              callbacks=[es, plateau],
              validation_batch_size=len(y_test),
              shuffle=True,
             verbose = 1)

    preds = model.predict([num_data_test]).reshape(1,-1)[0]
    score = round(roc_auc_score(y_test,preds),5)
    print('Fold {} {}: {}'.format(counter, model_name, score))
    scores_folds[model_name].append(score)
    
    tt =scaler.transform(test_nn[features_to_consider].values)
    
    test_predictions_nn += model.predict([tt]).reshape(1,-1)[0].clip(0,1e10)/n_folds       
    counter += 1

Training fold 1
CV 1/5
(480000, 100) (480000,)
(120000, 100) (120000,)
Epoch 1/1000
235/235 [==============================] - 4s 12ms/step - loss: 0.6933 - auc: 0.5014 - val_loss: 0.6931 - val_auc: 0.5000 - lr: 0.0010
Epoch 2/1000
235/235 [==============================] - 2s 10ms/step - loss: 0.6931 - auc: 0.4991 - val_loss: 0.6931 - val_auc: 0.5000 - lr: 0.0010
Epoch 3/1000
235/235 [==============================] - 2s 11ms/step - loss: 0.6927 - auc: 0.5338 - val_loss: 0.6907 - val_auc: 0.6899 - lr: 0.0010
Epoch 4/1000
235/235 [==============================] - 3s 11ms/step - loss: 0.6677 - auc: 0.7093 - val_loss: 0.6329 - val_auc: 0.7306 - lr: 0.0010
Epoch 5/1000
235/235 [==============================] - 3s 11ms/step - loss: 0.6158 - auc: 0.7262 - val_loss: 0.6051 - val_auc: 0.7329 - lr: 0.0010
Epoch 6/1000
235/235 [==============================] - 3s 11ms/step - loss: 0.6041 - auc: 0.7305 - val_loss: 0.6011 - val_auc: 0.7341 - lr: 0.0010
Epoch 7/1000
235/235 [===================

In [ ]:
test_predictions_nn.shape

(540000,)

In [ ]:
sub=pd.read_csv("/content/drive/MyDrive/빅데이터 분석 실습/[실전 과제] Kaggle playground/sample_submission.csv")
sub['target']=test_predictions_nn

In [ ]:
sub.to_csv("submission_2.csv",index=False)